In [ ]:
import networkx.algorithms.isomorphism as iso
import networkx as nx
import pandas as pd
import numpy as np

In [ ]:
def load_graph(file):
    data = file.values
    G = nx.Graph()
    
    for i in range(len(data)):
        G.add_edge(data[i,0], data[i,1])
    return G

In [ ]:
def load_labels(graph):
    for x in graph.nodes:
        graph.nodes[x]['label']=x

In [ ]:
dppin=pd.read_csv('../Data/dppintarrasov.txt')

In [ ]:
Gdppin = load_graph(dppin)

In [ ]:
load_labels(Gdppin)

In [ ]:
import signal
import time

def timeout_handler(num, stack):
    print("Received SIGALRM")
    raise Exception("FOREVER")

signal.signal(signal.SIGALRM, timeout_handler)

#need to iterate over all nodes in the graph, remove all matches to node each
def check_iso3(G):
    iso_groups={}
    set_to_check = set(G.nodes)
    print('nodes in set: ', len(set_to_check))
    for node1 in G.nodes:
        try:
            signal.alarm(3600)
            #signal.alarm(3)
            sub = nx.ego_graph(G, node1, radius = 3, undirected=True)
                #print(G.nodes[node1]['label'])
            print(node1)
            for node2 in G.nodes:
                #print(G.nodes[node2]['label'])
                if node2 in set_to_check:
                    sub2 = nx.ego_graph(G, node2, radius = 3, undirected=True)
                    if nx.is_isomorphic(sub, sub2):
                        if G.nodes[node1]['label'] not in iso_groups:
                            iso_groups[G.nodes[node1]['label']]=set()
                            #print("iso_groups after add: ", iso_groups)
                        iso_groups[G.nodes[node1]['label']].add(G.nodes[node2]['label'])
                        set_to_check.remove(node2)
                    #print("checked a single isomorphism!")
            signal.alarm(0)
        except Exception as ex:
            print("Too Long! Going to 2-hop neighborhood")
            sub = nx.ego_graph(G, node1, radius = 2, undirected=False)
            #print(G.nodes[node1]['label'])
            print(node1)
            for node2 in G.nodes:
                #print(G.nodes[node2]['label'])
                if node2 in set_to_check:
                    sub2 = nx.ego_graph(G, node2, radius = 2, undirected=False)
                    if nx.is_isomorphic(sub, sub2):
                        if G.nodes[node1]['label'] not in iso_groups:
                            iso_groups[G.nodes[node1]['label']]=set()
                            #print("iso_groups after add: ", iso_groups)
                        iso_groups[G.nodes[node1]['label']].add(G.nodes[node2]['label'])
                        set_to_check.remove(node2)
        finally:
            print('Back to 3-hop!')
        print("iso groups size : ", node1, len(iso_groups))
        print("nodes left in set: ", len(set_to_check))
                        
                    
    return iso_groups

In [ ]:
import time
start = time.time()
isos = check_iso3(Gdppin)
end = time.time()
total=end-start

In [ ]:
total

In [ ]:
# Automorphism group sizes
counts=[]
for x in isos:
    counts.append(len(isos[x]))

In [ ]:
# number of unique nodes
list(counts).count(1)

In [ ]:
440/905

In [ ]:
# size of largest automorphic group
np.array(counts).max()

In [ ]:
import matplotlib.pyplot as plt

f, (ax, ax2) = plt.subplots(2, 1, sharex=True)

# plot the same data on both axes
ax.hist(counts, bins=40, edgecolor='black')
ax2.hist(counts, bins=40, edgecolor='black')

# zoom-in / limit the view to different portions of the data
ax.set_ylim(100, 600)  
ax2.set_ylim(0, 20)  

# hide the spines between ax and ax2
ax.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax.xaxis.tick_top()
ax.tick_params(labeltop=False)  # don't put tick labels at the top
ax2.xaxis.tick_bottom()


d = .015  

kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
ax.plot((-d, +d), (-d, +d), **kwargs)        # top-left diagonal
ax.plot((1 - d, 1 + d), (-d, +d), **kwargs)  # top-right diagonal

kwargs.update(transform=ax2.transAxes)  # switch to the bottom axes
ax2.plot((-d, +d), (1 - d, 1 + d), **kwargs)  # bottom-left diagonal
ax2.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)  # bottom-right diagonal


#ax.set_title('Distribution of Sizes of Automorphic Groups')
plt.xlabel('Size of Automorphic Group (# nodes)')
plt.ylabel('Count')
plt.show()
#plt.savefig('Dppindisttrue2.pdf', bbox_inches='tight')